# Import Python module

In [1]:
import pandas as pd
import json
from datetime import datetime
from pymongo import MongoClient
import pprint

# Load cusotmer.csv

- Load the file into a Pandas dataframe. If load a large file on a machine with limited ram, consider use chunk
- rename the column to lower case
- print out dataframe information for debug

In [2]:
df_customer = pd.read_csv('../data_load/incoming/customer.csv', sep=',', header =0)
df_customer.rename(columns=lambda x: x.lower(), inplace=True)
df_customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   customer_id  1000 non-null   object
 1   first_name   1000 non-null   object
 2   last_name    1000 non-null   object
 3   gender       1000 non-null   object
 4   dob          1000 non-null   object
dtypes: object(5)
memory usage: 39.2+ KB


In [3]:
df_customer.head(5)

,customer_id,first_name,last_name,gender,dob
0,C000001,Marty,McFly,M,1970-01-01 00:00:00
1,C000002,Aurora,Lane,F,1970-01-02 00:00:00
2,C000003,Rita,Vrataski,F,1970-01-03 00:00:00
3,C000004,William,Cage,M,1970-01-04 00:00:00
4,C000005,Stanley,Ipkiss,M,1970-01-05 00:00:00


# Convert DOB to date type
we can do it when we load the data, just use a different way

In [4]:
df_customer['dob'] = df_customer['dob'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
df_customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   customer_id  1000 non-null   object        
 1   first_name   1000 non-null   object        
 2   last_name    1000 non-null   object        
 3   gender       1000 non-null   object        
 4   dob          1000 non-null   datetime64[ns]
dtypes: datetime64[ns](1), object(4)
memory usage: 39.2+ KB


# Connect to MongoDB
- The MongoDB replicaSet is running in a docker container(d2) in my laptop without authentication.
- Due to the jupyter notebook is hosted in d1, I can use d-rh7-2 to connect instead of 127.0.0.1.
- ocean_studio_III is just a name.

In [5]:
mongo_db= MongoClient(
    'mongodb://d-rh7-2:27017/?authSource=admin&replicaSet=rs_falcon',
    ssl=False,
    maxPoolSize = 5,
    wtimeout = 2500)['ocean_studio_III']

mongo_db.list_collection_names()
customer_collection = mongo_db['customer']

# Dump to MongoDB
- convert dataframe to json array
- insert them to MongoDB

In [6]:
customer_collection.delete_many({})
temp_json_result=df_customer.to_json(orient='records')
parsed_json = json.loads(temp_json_result)
mongodb_result = customer_collection.insert_many(parsed_json)
print(len(mongodb_result.inserted_ids))

1000


# Create Index
- Codify index creation process without undestanding what the impact is. 
- MongoDb development team should not rely on a *traditional* DBA to create index.
- These Index creation might be part of application bootstrap with additional intelligence, such as
    - Check if index was already created
    - Checke if collection has data in there. Consider to either abort or create index in background.

In [7]:
if 'customer_id_uk' not in list(customer_collection.index_information()):
    mongodb_result=customer_collection.create_index("customer_id", name='customer_id_uk', unique=True)
else:
    print('Already created')
    pprint.pprint(customer_collection.index_information())

Already created
{'_id_': {'key': [('_id', 1)], 'ns': 'ocean_studio_III.customer', 'v': 2},
 'customer_id_uk': {'key': [('customer_id', 1)],
                    'ns': 'ocean_studio_III.customer',
                    'unique': True,
                    'v': 2}}


In [8]:
pd.set_option('display.max_columns', 100) # show 100 columns
pd.set_option('display.max_rows', 100)
df_customer

,customer_id,first_name,last_name,gender,dob
0,C000001,Marty,McFly,M,1970-01-01
1,C000002,Aurora,Lane,F,1970-01-02
2,C000003,Rita,Vrataski,F,1970-01-03
3,C000004,William,Cage,M,1970-01-04
4,C000005,Stanley,Ipkiss,M,1970-01-05
...,...,...,...,...,...
995,C000996,First 000996,Last 000996,F,1972-09-23
996,C000997,First 000997,Last 000997,M,1972-09-24
997,C000998,First 000998,Last 000998,F,1972-09-25
998,C000999,First 000999,Last 000999,M,1972-09-26


# Search
- Search in dataframe
- Search in MongoDB

In [9]:
filt = (df_customer['first_name'] == 'William') & (df_customer['last_name'] == 'Cage')
df_customer[filt]

,customer_id,first_name,last_name,gender,dob
3,C000004,William,Cage,M,1970-01-04


In [10]:
for customer in customer_collection.find({"first_name": "William", "last_name": "Cage"}):
    pprint.pprint(customer)

{'_id': ObjectId('607252399b956a0ef037e8e1'),
 'customer_id': 'C000004',
 'dob': 259200000,
 'first_name': 'William',
 'gender': 'M',
 'last_name': 'Cage'}
